# Working

In [44]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from pandas.core.roperator import rpow
import os
import re

In [2]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Define data as a dictionary with keys as column names and values as lists
data = {
    'Image_Name': [],
    'Orig_Image_Name': [],
    'Category': [],
    'Alt': [],
    'Type': [],
    'Caption': [],
    'URL_Image': [],
    'URL_Info': []
}

# Create the DataFrame
df_indexed_images = pd.DataFrame(data)
df_indexed_images.drop(df_indexed_images.index, inplace=True)


# Print the empty DataFrame
print(df_indexed_images)

Empty DataFrame
Columns: [Image_Name, Orig_Image_Name, Category, Alt, Type, Caption, URL_Image, URL_Info]
Index: []


In [ ]:
df_indexed_images

,Image_Name,Orig_Image_Name,Category,Alt,Type,Caption,URL_Image,URL_Info


In [ ]:
def crawl_page(page_url, category):
  global loop_page_count
  global image_count
  print(str(loop_page_count))

  if loop_page_count == 1:
    url_1 = page_url
  else:
    url_1 = page_url + "&page=" + str(loop_page_count)

  print('--------------------------------------------------Crawling for page: ' + url_1)
  img_category = category

  os.chdir("/content/drive/MyDrive/CA6005I - Mechanics of Search/Assignment2/Code/images")

  response_1 = requests.get(url_1)

  # Parse HTML content
  soup_page_1 = BeautifulSoup(response_1.content, "html.parser")

  # Loop through all the divs with class "m-4"
  for div in soup_page_1.find_all("div", class_="m-4"):
      # Find the image tag inside this div
      img = div.find("img")

      # Check if the alt attribute of the image starts with "Image of"
      if img and img.get("alt", "").lower().startswith('image of'):
          # If so, print the src attribute and the class of the paragraph containing the image
          img_alt = img["alt"].replace("Image of ","").strip()
          #print("img_alt: " + img_alt)
          img_src = img["src"].strip()
          img_url = "http://worldimages.sjsu.edu" + img_src
          img_type = div.find('p', {'class': 'card-text small'}).text.strip()
          #img_type = div.find('a', {'class': 'noColor'}).text.strip()
          img_type = img_type.strip().split('\n')[-1].strip()

          atag = div.find('a')
          href = atag['href']

          # Send a GET request to the web page
          url_2 = 'http://worldimages.sjsu.edu' + href

          response_2 = requests.get(url_2)
          
          # Parse the HTML content using Beautiful Soup
          soup_2 = BeautifulSoup(response_2.content, 'html.parser')

          # Find the div tag with class "object-info-field" and extract its text content
          div_tag = soup_2.find('div', class_='object-info-field')
          div_text = div_tag.text.replace("\n", "").replace("\r", "").replace("  ", "").strip()
          img_caption = div_text.replace("Caption:","")

          orig_filename = img_url.split('/')[-1]
          img_index = image_count + 1
          img_index_str = str(img_index).zfill(4)

          indexed_filename = "image_" + img_index_str
          with open(indexed_filename, 'wb') as f:
            f.write(requests.get(img_url).content)

          print('indexed_filename: ' + indexed_filename)
          # print('orig_filename: ' + orig_filename)
          # print('img_category: ' + img_category)
          # print('img_alt: ' + img_alt)
          # print('img_type: ' + img_type)
          # print('img_caption: ' + img_caption)
          # print('url_1: ' + url_1)
          # print('url_2: ' + url_2)
          # print("\n")

          df_indexed_images.loc[image_count, 'Image_Name'] = indexed_filename
          df_indexed_images.loc[image_count, 'Orig_Image_Name'] = orig_filename
          df_indexed_images.loc[image_count, 'Category'] = img_category
          df_indexed_images.loc[image_count, 'Alt'] = img_alt
          df_indexed_images.loc[image_count, 'Type'] = img_type
          df_indexed_images.loc[image_count, 'Caption'] = img_caption
          df_indexed_images.loc[image_count, 'URL_Image'] = url_1
          df_indexed_images.loc[image_count, 'URL_Info'] = url_2

          image_count += 1

In [ ]:
loop_page_count = 0
image_count = 0

In [ ]:
image_count

1074

In [ ]:
for page in range(18):
  loop_page_count += 1
  print("loop_page_count: " + str(loop_page_count))
  print("image_count: " + str(image_count))
  crawl_page("http://worldimages.sjsu.edu/objects-1/portfolio?records=12&query=Portfolios%20%3D%20%222206%22&sort=0","Protest")

In [ ]:
df_indexed_images

,Image_Name,Orig_Image_Name,Category,Alt,Type,Caption,URL_Image,URL_Info
0,image_0001,turi9088.png,Warfare: World War II,Mussolini on cover of 'La Domenica dell' Agric...,20th c Italian Journalistic Document,ITALIAN Anonymous | Mussolini on cover of 'La ...,http://worldimages.sjsu.edu/objects-1/portfoli...,http://worldimages.sjsu.edu/objects-1/info?que...
1,image_0002,euh07084.png,Warfare: World War II,Mussolini Mementos Window; Messina.,20th c Italian Applied Arts,FASCIST | Mussolini Mementos Window; Messina. ...,http://worldimages.sjsu.edu/objects-1/portfoli...,http://worldimages.sjsu.edu/objects-1/info?que...
2,image_0003,wash0076.png,Warfare: World War II,Youthful members of the Balillas (Junior Fasci...,20th c Italian Photography,ITALIAN | Youthful members of the Balillas (Ju...,http://worldimages.sjsu.edu/objects-1/portfoli...,http://worldimages.sjsu.edu/objects-1/info?que...
3,image_0004,ethi4030.png,Warfare: World War II,St. George Cathedral. Heili Selassi and his ar...,20th c Ethiopian Realist Painting,ANDELO Alec | St. George Cathedral. Heili Sela...,http://worldimages.sjsu.edu/objects-1/portfoli...,http://worldimages.sjsu.edu/objects-1/info?que...
4,image_0005,war01047.png,Warfare: World War II,Man in Ruins.,20th c German Expressionist Painting,HOFFER Karl | Man in Ruins. | 1937 | German | ...,http://worldimages.sjsu.edu/objects-1/portfoli...,http://worldimages.sjsu.edu/objects-1/info?que...
...,...,...,...,...,...,...,...,...
1279,image_1280,vien3758.png,Protest,Rebellious Silence. Women of Allah.,20th c Iranian (Persian) Post Modern Photography,NESHAT Shirin (b. 1957 Iran)| Rebellious Silen...,http://worldimages.sjsu.edu/objects-1/portfoli...,http://worldimages.sjsu.edu/objects-1/info?que...
1280,image_1281,nymt2180.png,Protest,New York City 1969.,20th c American Documentary Photography,WINOGRAD Gary | New York City 1969.| 1969 | Am...,http://worldimages.sjsu.edu/objects-1/portfoli...,http://worldimages.sjsu.edu/objects-1/info?que...
1281,image_1282,wom04076.png,Protest,Pat Whalen.,20th c American Expressionist Painting,NEEL Alice | Pat Whalen. | 1935 | American | E...,http://worldimages.sjsu.edu/objects-1/portfoli...,http://worldimages.sjsu.edu/objects-1/info?que...
1282,image_1283,wom04077.png,Protest,Pat Whalen. Detail.,20th c American Expressionist Painting,NEEL Alice | Pat Whalen. Detail. | 1935 | Amer...,http://worldimages.sjsu.edu/objects-1/portfoli...,http://worldimages.sjsu.edu/objects-1/info?que...


In [ ]:
os.chdir("/content/drive/MyDrive/CA6005I - Mechanics of Search/Assignment2/Code/indexation")
!ls

Indexed_Images.csv


In [ ]:
df_indexed_images.to_csv('Indexed_Images_Raw.csv', index=False)

## Preprocess index

In [34]:
os.chdir("/content/drive/MyDrive/CA6005I - Mechanics of Search/Assignment2/Code/indexation")
!ls

Indexed_Images.gsheet  Indexed_Images_Preprocessed.csv	Indexed_Images_Raw.csv


In [35]:
df_indexed_images =  pd.read_csv("Indexed_Images_Raw.csv")

In [36]:
# create a new column "Image_Details" by concatenating values from the 4 columns
df_indexed_images["Image_Details"] = df_indexed_images["Category"].fillna('') + " " + \
                                     df_indexed_images["Alt"].fillna('') + " " + \
                                     df_indexed_images["Type"].fillna('') + " " + \
                                     df_indexed_images["Caption"].fillna('')

# drop the 4 columns that were merged
df_indexed_images.drop(columns=["Category", "Alt", "Type", "Caption"], inplace=True)

In [37]:
# replace all occurrences of " |" with ""
df_indexed_images["Image_Details"] = df_indexed_images["Image_Details"].str.replace(" \|", "")

<ipython-input-37-e84532cc48e3>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df_indexed_images["Image_Details"] = df_indexed_images["Image_Details"].str.replace(" \|", "")


In [39]:
# replace all occurrences of " |" with ""
df_indexed_images["Image_Details"] = df_indexed_images["Image_Details"].str.replace("|", "")

<ipython-input-39-a8ccd342e524>:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_indexed_images["Image_Details"] = df_indexed_images["Image_Details"].str.replace("|", "")


In [45]:
# Define a regular expression to match unwanted characters
unwanted_chars_regex = r'[^\w\d\s]'

# Define a regular expression to match non-ASCII characters
non_ascii_regex = r'[^\x00-\x7F]'

# Replace unwanted characters and non-ASCII characters with empty strings
df_indexed_images["Image_Details"] = df_indexed_images["Image_Details"].str.replace(unwanted_chars_regex, '').str.replace(non_ascii_regex, '')

# Convert all text in the "Image_Details" column to lowercase
df_indexed_images["Image_Details"] = df_indexed_images["Image_Details"].str.lower()


<ipython-input-45-0f4024d59d78>:8: FutureWarning: The default value of regex will change from True to False in a future version.
  df_indexed_images["Image_Details"] = df_indexed_images["Image_Details"].str.replace(unwanted_chars_regex, '').str.replace(non_ascii_regex, '')


In [51]:
df_indexed_images

,Image_ID,Image_Name,Orig_Image_Name,URL_Image,URL_Info,Image_Details
0,1,image_0001,turi9088.png,http://worldimages.sjsu.edu/objects-1/portfoli...,http://worldimages.sjsu.edu/objects-1/info?que...,warfare world war ii mussolini on cover of la ...
1,2,image_0002,euh07084.png,http://worldimages.sjsu.edu/objects-1/portfoli...,http://worldimages.sjsu.edu/objects-1/info?que...,warfare world war ii mussolini mementos window...
2,3,image_0003,wash0076.png,http://worldimages.sjsu.edu/objects-1/portfoli...,http://worldimages.sjsu.edu/objects-1/info?que...,warfare world war ii youthful members of the b...
3,4,image_0004,ethi4030.png,http://worldimages.sjsu.edu/objects-1/portfoli...,http://worldimages.sjsu.edu/objects-1/info?que...,warfare world war ii st george cathedral heili...
4,5,image_0005,war01047.png,http://worldimages.sjsu.edu/objects-1/portfoli...,http://worldimages.sjsu.edu/objects-1/info?que...,warfare world war ii man in ruins 20th c germa...
...,...,...,...,...,...,...
1279,1280,image_1280,vien3758.png,http://worldimages.sjsu.edu/objects-1/portfoli...,http://worldimages.sjsu.edu/objects-1/info?que...,protest rebellious silence women of allah 20th...
1280,1281,image_1281,nymt2180.png,http://worldimages.sjsu.edu/objects-1/portfoli...,http://worldimages.sjsu.edu/objects-1/info?que...,protest new york city 1969 20th c american doc...
1281,1282,image_1282,wom04076.png,http://worldimages.sjsu.edu/objects-1/portfoli...,http://worldimages.sjsu.edu/objects-1/info?que...,protest pat whalen 20th c american expressioni...
1282,1283,image_1283,wom04077.png,http://worldimages.sjsu.edu/objects-1/portfoli...,http://worldimages.sjsu.edu/objects-1/info?que...,protest pat whalen detail 20th c american expr...


In [49]:
# Reset the index of the DataFrame and add 1 to create a new "id" column at the beginning
df_indexed_images.insert(0, 'Image_ID', df_indexed_images.reset_index().index + 1)

In [52]:
df_indexed_images.to_csv('Indexed_Images_Preprocessed.csv', index=False)